In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import tensorcircuit as tc
import jax
from jax import random
import tqdm
import numpy as np
import optax
import pandas as pd
import matplotlib.pyplot as plt
import time

from queso.io import IO

backend = tc.set_backend("jax")
tc.set_dtype("complex128")
tc.set_contractor("auto")  # “auto”, “greedy”, “branch”, “plain”, “tng”, “custom”

functools.partial(<function custom at 0x1360ded30>, optimizer=<function auto at 0x1351c1430>, memory_limit=None, debug_level=0)

In [4]:
def cnot_2local_ansatz(params, n, k):
    c = tc.Circuit(n)
    for i in range(k):
        for j in range(n):
            c.r(
                j,
                theta=params[3 * j, i],
                alpha=params[3 * j + 1, i],
                phi=params[3 * j + 2, i],
            )

        for j in range(1, n, 2):
            c.cnot(j - 1, j)

        for j in range(2, n, 2):
            c.cnot(j - 1, j)
    return c

In [5]:
n = 15
k = 4
n_samples = 1000
ansatz = "cnot_2local_ansatz"
seed = 0

In [6]:
shape = [3 * n, k]
key = random.PRNGKey(seed)
theta = random.uniform(key, shape, minval=0, maxval=2*np.pi)

circ = cnot_2local_ansatz(theta, n, k)

In [7]:
x, p = zip(*circ.sample(batch=10))
shots = np.stack(x).astype("int8")
print(shots)

[[0 0 1 0 0 0 1 0 0 1 0 1 1 0 1]
 [1 0 1 1 0 1 1 1 1 1 1 0 0 0 1]
 [0 1 1 1 1 0 0 1 1 0 0 1 1 0 1]
 [0 0 0 0 1 1 0 0 1 0 0 0 1 1 1]
 [0 0 0 0 1 1 0 1 1 0 1 0 1 0 0]
 [1 0 0 1 1 0 0 0 1 0 1 1 1 1 0]
 [1 1 0 0 0 1 0 0 1 0 1 1 0 1 1]
 [0 1 0 1 1 0 1 1 0 0 1 0 1 1 0]
 [0 1 0 1 0 1 0 0 1 0 0 1 0 1 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 1 0]]


In [9]:
import quimb